## Wprowadzenie
Skrypt pokazuje jak użyć pakietu SciKit do klasyfikacji danych. Rozważane są dwa przykłady:  zestaw danych IRIS oraz zestaw danych TITANIC  (do ściągnięcia z https://www.kaggle.com/c/titanic, dokładniej: potrzebny jest  plik https://www.kaggle.com/c/titanic/download/train.csv).

In [1]:
% matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import tree

## 1. Pierwszy zestaw danych
Dane IRIS

In [2]:
# wczytanie zestawu danych
from sklearn import datasets
iris = datasets.load_iris()

data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
# rozbicie zestawu danych na dane opisujące kwiat (X) i etykietę klasy (y)
y = data['species']
X = data.drop('species', axis = 1)

# stworzenie drzewa klasyfikacyjnego
t = tree.DecisionTreeClassifier()
t = t.fit(X, y)

In [4]:
# zapisanie drzewa klasyfikacyjnego do pliku .dot
# plik ten można przekształcić do pliku .pdf za pomocą programu graphviz używając polecenia:
#   dot -Tpdf iris.dot -o iris.pdf

with open("output/iris.dot", "w") as f:
    tree.export_graphviz(t, out_file=f, feature_names=X.columns)

In [5]:
# ocena stworzonego klasyfikatora na danych uczących
t.score(X, y)

1.0

In [6]:
# Uczciwiej byłoby oceniać klasyfikator na danych, które nie były używane podczas tworzenia
# klasyfikatora. Dlatego cały zestaw danych warto podzielić na dwie części: dane uczące i dane
# testowe.

data['train'] = np.random.uniform(0, 1, len(data))

data_train = data[data['train'] <= 0.65]
data_test = data[data['train'] > 0.65]

y = data_train['species']
X = data_train.drop('species', axis = 1)

t = tree.DecisionTreeClassifier()
t = t.fit(X, y)

print(t.score(X, y))

y = data_test['species']
X = data_test.drop('species', axis = 1)

print(t.score(X, y))

1.0
0.9692307692307692


## 2. Drugi zestaw danych
Dane TITANIC (do ściągnięcia z https://www.kaggle.com/c/titanic, dokładniej: potrzebny jest  plik https://www.kaggle.com/c/titanic/download/train.csv).

In [7]:
# wczytanie zestawu danych z pliku
data = pd.read_csv("data/titanic.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# usunięcie z zestawu danych atrybutów nieistotnych dla klasyfikacji
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
data = data.dropna()
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [9]:
# zmiana kodowania atrybutów nominalnych w zestawie danych
data['Sex'] = pd.Categorical(data['Sex']).codes
data['Embarked'] = pd.Categorical(data['Embarked']).codes
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


# a) 

Domyślnie indeks Giniego, ale można też wybrać entropię.

In [10]:
# rozbicie zestawu danych na dane opisujące pasażera (X) i etykietę klasy (y)
y = data['Survived']
X = data.drop('Survived', axis = 1)

# stworzenie drzewa klasyfikacyjnego
tGini = tree.DecisionTreeClassifier()
tGini = tGini.fit(X, y)

tEnt = tree.DecisionTreeClassifier(criterion='entropy')
tEnt = tEnt.fit(X, y)

In [11]:
# zapisanie drzewa klasyfikacyjnego do pliku .dot
# plik ten można przekształcić do pliku .pdf za pomocą programu graphviz używając polecenia:
#   dot -Tpdf titanic.dot -o titanic.pdf

with open("output/titanicGini.dot", "w") as f:
    tree.export_graphviz(tGini, out_file=f, feature_names=X.columns)
    
with open("output/titanicEnt.dot", "w") as f:
    tree.export_graphviz(tEnt, out_file=f, feature_names=X.columns)

In [12]:
# ocena stworzonego klasyfikatora na danych uczących
print(tGini.score(X, y))
print(tEnt.score(X, y))
      

0.9859550561797753
0.9859550561797753


# b)

In [13]:
# Uczciwiej byłoby oceniać klasyfikator na danych, które nie były używane podczas tworzenia
# klasyfikatora. Dlatego cały zestaw danych warto podzielić na dwie części: dane uczące i dane
# testowe (lista 5 zadanie 2b).
data['train'] = np.random.uniform(0, 1, len(data))

data_train = data[data['train'] <= 0.65]
data_test = data[data['train'] > 0.65]

y = data_train['Survived']
X = data_train.drop('Survived', axis = 1)


tGini = tree.DecisionTreeClassifier()
tGini = tGini.fit(X, y)

tEnt = tree.DecisionTreeClassifier(criterion='entropy')
tEnt = tEnt.fit(X, y)

print(tGini.score(X, y))
print(tEnt.score(X, y))

y = data_test['Survived']
X = data_test.drop('Survived', axis = 1)

print(tGini.score(X, y))
print(tEnt.score(X, y))

1.0
1.0
0.39147286821705424
0.5348837209302325


# c)

In [14]:
print(tGini.tree_.max_depth)
print(tEnt.tree_.max_depth)

15
18


In [15]:
for i in range(5, 15, 2):
    data['train'] = np.random.uniform(0, 1, len(data))

    data_train = data[data['train'] <= 0.65]
    data_test = data[data['train'] > 0.65]

    y = data_train['Survived']
    X = data_train.drop('Survived', axis = 1)


    tGini = tree.DecisionTreeClassifier(max_depth=i)
    tGini = tGini.fit(X, y)


    tEnt = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i)
    tEnt = tEnt.fit(X, y)
    print("\ni: ", i)
    print('Train Gini: \t', tGini.score(X, y))
    print('Train Ent: \t',tEnt.score(X, y))

    y = data_test['Survived']
    X = data_test.drop('Survived', axis = 1)

    print('Test Gini: \t',tGini.score(X, y))
    print('Test Ent: \t',tEnt.score(X, y))


i:  5
Train Gini: 	 0.8742004264392325
Train Ent: 	 0.8784648187633263
Test Gini: 	 0.7818930041152263
Test Ent: 	 0.7860082304526749

i:  7
Train Gini: 	 0.9209401709401709
Train Ent: 	 0.9252136752136753
Test Gini: 	 0.7459016393442623
Test Ent: 	 0.7704918032786885

i:  9
Train Gini: 	 0.9215686274509803
Train Ent: 	 0.9281045751633987
Test Gini: 	 0.7351778656126482
Test Ent: 	 0.83399209486166

i:  11
Train Gini: 	 0.9710467706013363
Train Ent: 	 0.9576837416481069
Test Gini: 	 0.7262357414448669
Test Ent: 	 0.7376425855513308

i:  13
Train Gini: 	 0.995475113122172
Train Ent: 	 0.9773755656108597
Test Gini: 	 0.6851851851851852
Test Ent: 	 0.7518518518518519


# d)

In [16]:

data['train'] = np.random.uniform(0, 1, len(data))

data_train = data[data['train'] <= 0.65]
data_test = data[data['train'] > 0.65]

y = data_train['Survived']
X = data_train.drop('Survived', axis = 1)


tGini = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=16, min_samples_leaf=8)
tGini = tGini.fit(X, y)


tEnt = tree.DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_split=16, min_samples_leaf=8)
tEnt = tEnt.fit(X, y)
print("\ni: ", i)
print('Train Gini: \t', tGini.score(X, y))
print('Train Ent: \t',tEnt.score(X, y))



y = data_test['Survived']
X = data_test.drop('Survived', axis = 1)

print('Test Gini: \t',tGini.score(X, y))
print('Test Ent: \t',tEnt.score(X, y))


i:  13
Train Gini: 	 0.8768898488120951
Train Ent: 	 0.8725701943844493
Test Gini: 	 0.7710843373493976
Test Ent: 	 0.7670682730923695


# e)

In [17]:
def titanicCrossValidate(data, model, name=''):
    data['train'] = (np.random.uniform(0,1, len(data)) * 10).astype(int)
    err = 0
    print(f"\n*** {name} ***")
    for i in range(10):
        data_train = data[data['train'] != i]
        data_test = data[data['train'] == i]
        
        y_train = data_train['Survived']
        X_train = data_train.drop('Survived', axis = 1)
        model.fit(X_train, y_train)
        
        y_test = data_test['Survived']
        X_test = data_test.drop('Survived', axis = 1)
        
        err += model.score(X_test, y_test)
#         print(f'\t{model.score(X_test, y_test)}')
    print(err/10)
    return err

In [18]:
models = [
    (tree.DecisionTreeClassifier(), 'Gini default'),
    (tree.DecisionTreeClassifier(criterion='entropy'), 'Entropy default'),
    (tree.DecisionTreeClassifier(max_depth=9), 'Gini, max_depth = 9'),
    (tree.DecisionTreeClassifier(criterion='entropy', max_depth=9), 'Entropy, max_depth = 9'),
    (
        tree.DecisionTreeClassifier(max_depth=7, min_samples_split=16, min_samples_leaf=8), 
        'Gini, max_depth = 9, min_samples_split = 16, min_samples_leaf = 8'
    ),
    (
        tree.DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_split=16, min_samples_leaf=8),
        'Entropy, max_depth = 9, min_samples_split = 16, min_samples_leaf = 8'
    )
]

In [19]:
for model, name in models:
    titanicCrossValidate(data, model, name)


*** Gini default ***
0.7504595818738656

*** Entropy default ***
0.7630843465935913

*** Gini, max_depth = 9 ***
0.7559372729120015

*** Entropy, max_depth = 9 ***
0.7835942631158017

*** Gini, max_depth = 9, min_samples_split = 16, min_samples_leaf = 8 ***
0.8182552073318036

*** Entropy, max_depth = 9, min_samples_split = 16, min_samples_leaf = 8 ***
0.8023882273516479
